In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Load and configure webdriver.
options = Options()
# Stop browser windows from actually popping up.
options.add_argument('--headless')
# Install a browser for use by Selenium.
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [7]:
# Navigate to US Chess tournament search page.
USCF_URL = 'http://www.uschess.org/datapage/events-rated.php'
uscf_events_page = driver.get(USCF_URL)

In [17]:
# Select a month on the page.
date_to_visit = '11/2022'
date_search_box = driver.find_element('name', 'month')
date_search_box.send_keys(date_to_visit)

# Select CA as the State Code, which is where all 
# chess.com USCF tournaments are registered.
state_search_box = driver.find_element('name', 'states')
state_search_box.send_keys('CA')
state_search_box.submit()